<a href="https://colab.research.google.com/github/Sanjaya-Shastry/tarantino-LLM-NN-/blob/main/LLM_0(tarantino).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/Sanjaya-Shastry/tarantino-LLM-NN-/refs/heads/main/scripts.txt

--2025-07-13 11:23:49--  https://raw.githubusercontent.com/Sanjaya-Shastry/tarantino-LLM-NN-/refs/heads/main/scripts.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 634257 (619K) [text/plain]
Saving to: ‘scripts.txt.5’

scripts.txt.5       100%[===================>] 619.39K  --.-KB/s    in 0.06s   

2025-07-13 11:23:49 (10.8 MB/s) - ‘scripts.txt.5’ saved [634257/634257]



In [ ]:
import numpy as np
from tensorflow import keras
from keras import layers

In [ ]:
with open('scripts.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
text = text.lower()

In [ ]:
lines = text.splitlines()
cleaned_lines = [line.strip() for line in lines if line.strip() != '']
cleaned_text = ' '.join(cleaned_lines)

In [31]:
cleaned_text[:100]

'django unchained written by quentin tarantino i ext  countryside  broiling hot day as the films open'

In [ ]:
import string

allowed_chars = set(string.ascii_letters + string.digits + " .,")
cleaned_text= ''.join(char for char in cleaned_text if char in allowed_chars)

In [32]:
cleaned_text[:100]

'django unchained written by quentin tarantino i ext  countryside  broiling hot day as the films open'

In [ ]:
#tokenization

chars = sorted(set(cleaned_text))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for ch, i in char_to_int.items()}
vocab_size = len(chars)

In [ ]:
encoded_text = np.array([char_to_int[i] for i in cleaned_text], dtype=np.int32)

In [ ]:
len(encoded_text)

361467

In [ ]:
context_size=16
X=[]
y=[]

for i in range(len(encoded_text) - context_size):
    X.append(encoded_text[i:i+context_size])
    y.append(encoded_text[i+context_size])


In [ ]:
X= np.array(X)
y= np.array(y)

In [ ]:
#We want to convert these into vectors of dimension of len(vocab_size) i.e 39
#embeddings

X_embedded = np.zeros((len(X), context_size, vocab_size), dtype=np.float32)
y_embedded = np.zeros((len(X), vocab_size), dtype=np.float32)

for i in range(len(X)):
    for t in range(context_size):
        word_index = X[i][t]
        X_embedded[i, t, word_index] = 1
    y_embedded[i, y[i]] = 1


In [ ]:
X_flat = X_embedded.reshape(len(X), context_size * vocab_size)


In [ ]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Input(shape=(context_size * vocab_size,)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(vocab_size, activation='softmax')
    ])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_flat, y_embedded, batch_size=128, epochs=10, validation_split=0.25)


Epoch 1/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 39s 18ms/step - accuracy: 0.3615 - loss: 2.2170 - val_accuracy: 0.4287 - val_loss: 1.9095
Epoch 2/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 38s 17ms/step - accuracy: 0.5191 - loss: 1.5914 - val_accuracy: 0.4606 - val_loss: 1.8252
Epoch 3/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 41s 17ms/step - accuracy: 0.5721 - loss: 1.3977 - val_accuracy: 0.4664 - val_loss: 1.8204
Epoch 4/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.6073 - loss: 1.2727 - val_accuracy: 0.4650 - val_loss: 1.8662
Epoch 5/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 41s 16ms/step - accuracy: 0.6365 - loss: 1.1708 - val_accuracy: 0.4642 - val_loss: 1.9167
Epoch 6/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 32s 15ms/step - accuracy: 0.6614 - loss: 1.0846 - val_accuracy: 0.4591 - val_loss: 1.9738
Epoch 7/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 33s 16ms/step - accuracy: 0.6839 - loss: 1.0081 - val_accuracy: 0.4560 - val_loss: 2.0558
Epoch 8/10
2118/2118 ━━━━━━━━━━━━━━━━━━━━ 40s 15ms/step - accuracy: 0.7014 -

In [ ]:
model.save("LLM_0(tarantino).keras")

In [ ]:
def generate_text(seed, length=300):
    result = seed

    for i in range(length):
        cleaned_seed = ''.join([c for c in result.lower() if c in char_to_int])
        seed_len = len(cleaned_seed)

        if seed_len > context_size:
            context = cleaned_seed[-context_size:]
        elif seed_len < context_size:
            padding = ' ' * (context_size - seed_len)
            context = padding + cleaned_seed
        else:
            context = cleaned_seed

        input_seq = [char_to_int.get(c, 0) for c in context]

        x = np.zeros((1, context_size, vocab_size), dtype=np.float32)
        for j, char_index in enumerate(input_seq):
            x[0, j, char_index] = 1

        x_flat = x.reshape(1, context_size * vocab_size)
        preds = model.predict(x_flat, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = int_to_char[next_index]
        result += next_char

    return result


In [28]:
generate_text('Broomhilda was',length=50)

'Broomhilda was a proat of a find them. she white men start with '